In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/ADM/HW5/")

In [3]:
import networkx as nx
from tqdm import tqdm

#Read data

In [110]:
if True*False:
  print("casa")

In [4]:
def namestr(obj, namespace):
    return([name for name in namespace if namespace[name] is obj][0])

In [111]:
def create_graph_layer(G,file_type,file_name,include_self_loops,maximum_time):
  
  for line in tqdm(file_type):
    
    #Read the line
    splitted_line=line.split(" ")
    #Extract source node,destination node and interaction time
    source_node=int(splitted_line[0])
    destination_node=int(splitted_line[1])
    interaction_time=int(int(splitted_line[2])/3600)

    #Disregard self loops if required and take only first maximum_time hours of the dataset
    if ((source_node!=destination_node) or include_self_loops) and interaction_time<maximum_time:
      
      #If G has already an edge, increase its weight and update the interaction time
      if G.has_edge(source_node,destination_node) and file_name in G[source_node][destination_node]:
        G[source_node][destination_node][file_name][file_name]+=1
        G[source_node][destination_node][file_name]["time"]=interaction_time
      
      #If that edge is noy yet existing, initialize its type and value depending on the file we are reading
      else:
        if file_name=="a2q":
          G.add_edge(source_node, destination_node,key=file_name, a2q=1,time=interaction_time)
        if file_name=="c2q":
          G.add_edge(source_node, destination_node,key=file_name, c2q=1,time=interaction_time)
        if file_name=="c2a":
          G.add_edge(source_node, destination_node,key=file_name, c2a=1,time=interaction_time)
  return(G)

In [116]:
#Open dataset files
a2q=open("a2q.txt")
c2a=open("c2a.txt")
c2q=open("c2q.txt")

#Create multidirected graph
G = nx.MultiDiGraph()
G=create_graph_layer(G,c2a,"c2a",include_self_loops=False,maximum_time=1000)
G=create_graph_layer(G,a2q,"a2q",include_self_loops=False,maximum_time=1000)
G=create_graph_layer(G,c2q,"c2q",include_self_loops=False,maximum_time=1000)

#Close dataset files
a2q.close()
c2a.close()
c2q.close()

3it [00:00, 2684.07it/s]
3it [00:00, 9032.96it/s]
3it [00:00, 8267.35it/s]
